**Закрытый API - экономическая свобода стран**

In [ ]:
!pip install -q knoema

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.0 MB/s eta 0:00:00


In [ ]:
import knoema
import pandas as pd

knoema.app_id = "C2lyoUE"
knoema.app_secret = "2wLTUyzrqt26uw"

DATASET = "HFWSJIEF2016" # dataset ID - определили через внутренний код на сайте

In [ ]:
df = knoema.get(DATASET, Indicator="Overall Score")

if isinstance(df.columns, pd.MultiIndex):
    df.columns = [" | ".join([str(x) for x in col if str(x) != ""]) for col in df.columns]
# проходим по каждому многоуровневому кортежа
# превращаем каждый элемент кортежа в строку и убираем пустые значения
# объединяем уровни в одну строку через разделитель
else:
    df.columns = [str(c) for c in df.columns]
value_cols = [c for c in df.columns if "Overall Score" in c]

In [ ]:
idx_name = df.index.name or "Time" # преобразование значения индекса в дату, извлекаем только год
years = pd.to_datetime(df.index, errors="coerce").year
df_wide = df[value_cols].copy()
df_wide.insert(0, "heritage_year", years)

# с помощью melt() получаем страны как строки, которые раньше были столбцами
long = df_wide.melt(id_vars = "heritage_year",
                    var_name = "col",
                    value_name = "economic_freedom_score")

# "Country - Overall Score - A"
parts = long["col"].str.split(" \\| ", n=2, expand=True)
long["country"] = parts[0]
# первая часть = "Overall Score" - нужно оставить, вторую часть не учитываем и не берем
long = long.drop(columns=["col"])

In [ ]:
long["heritage_year"] = pd.to_numeric(long["heritage_year"], errors="coerce")
long["economic_freedom_score"] = pd.to_numeric(long["economic_freedom_score"], errors="coerce")
long = long.dropna(subset=["economic_freedom_score", "heritage_year", "country"])

# берём последнее доступное значение на страну
df_latest = (long.sort_values(["country","heritage_year"])
                  .drop_duplicates("country", keep="last")
                  .sort_values("country")
                  .reset_index(drop=True)
                  [["country","heritage_year","economic_freedom_score"]])

In [ ]:
df_latest.to_csv("heritage_index_knoema.csv", index=False)